In [2]:
# Relevant Reference Links for Stance Prediction
# For Queries q and Documents d at hand.
# Baseline Code: https://github.com/dh1105/Sentence-Entailment
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# execute once: Removing the extra directory.
!rm -r sample_data/

In [4]:
# loading the stance prediction dataset tsv file.
import pandas as pd
stance_df = pd.read_csv('/content/drive/MyDrive/touche-2022-prototyping/touche22-task2-stance-dataset.tsv', sep='\t')

In [5]:
print(set(stance_df.ds))

{'superuser.stackexchange', 'music.stackexchange', 'travel.stackexchange', 'gardening.stackexchange', 'cooking.stackexchange', 'cs.stackexchange', 'yahoo', 'academia.stackexchange', 'softwarerecs.stackexchange', 'softwareengineering.stackexchange'}


In [6]:
stance_df.head()

,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,softwareengineering.stackexchange,10373,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",2,simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,softwareengineering.stackexchange,18838,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",2,Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,softwareengineering.stackexchange,19392,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,3,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,softwareengineering.stackexchange,20653,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",3,expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,softwareengineering.stackexchange,21186,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",1,Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"


In [7]:
stance_df[stance_df.ds.isin(['yahoo'])]
# Removing 'yahoo' data source from the analysis.

,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
35,yahoo,1020731,1020731,1020731,3,Accord,2,Camry,"[[95, 100], [347, 352]]",Accord,"[[4, 10], [29, 35], [221, 227], [358, 364], [4..."
36,yahoo,1067597,1067597,1067597,3,LCD big screen tvs,2,plasmaq,"[[47, 54]]",LCD big screen tvs,[]
37,yahoo,120337,120337,120337,3,charcoal,2,gas,"[[66, 76], [168, 171], [1062, 1065], [1083, 10...",charcoal,"[[6, 14], [53, 61], [80, 88], [212, 227], [259..."
38,yahoo,1205525,1205525,1205525,1,Neutral,2,windows,"[[78, 85], [181, 188], [194, 201], [950, 957]]",linux,"[[359, 364], [685, 690], [876, 881]]"
39,yahoo,1211167,1211167,1211167,2,BMW,2,BMW,"[[174, 183], [249, 252], [441, 444], [493, 498...",infinity G35,"[[146, 165], [230, 238], [348, 356], [768, 780..."
...,...,...,...,...,...,...,...,...,...,...,...
951,yahoo,4268101,4268101,4268101,2,free downloadable browsers,2,free downloadable browsers,"[[73, 80], [227, 234], [480, 487], [739, 746]]",Internet Explorer,"[[775, 777]]"
952,yahoo,4369635,4369635,4369635,1,Neutral,2,second hand smoke,"[[149, 166]]",first hand,[]
953,yahoo,4413933,4413933,4413933,2,Rock,2,Rock,"[[10, 14], [472, 476], [500, 504]]",Rap,"[[23, 34], [572, 575]]"
954,yahoo,618287,618287,618287,3,Ang Lee,2,Tsui Hark,[],Ang Lee,"[[9, 16]]"


In [8]:
stance_df = stance_df[stance_df.ds.isin(['cs.stackexchange', 'superuser.stackexchange',
                                         'travel.stackexchange', 'gardening.stackexchange',
                                         'softwareengineering.stackexchange', 'softwarerecs.stackexchange',
                                         'music.stackexchange', 'academia.stackexchange', 'cooking.stackexchange'])]

In [9]:
print(set(stance_df.ds))
stance_df.head()

{'superuser.stackexchange', 'music.stackexchange', 'travel.stackexchange', 'gardening.stackexchange', 'cooking.stackexchange', 'cs.stackexchange', 'academia.stackexchange', 'softwarerecs.stackexchange', 'softwareengineering.stackexchange'}


,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,softwareengineering.stackexchange,10373,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",2,simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,softwareengineering.stackexchange,18838,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",2,Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,softwareengineering.stackexchange,19392,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,3,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,softwareengineering.stackexchange,20653,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",3,expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,softwareengineering.stackexchange,21186,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",1,Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"


In [10]:
from sklearn.utils import shuffle
dataset_df = shuffle(stance_df[['question','answer','answer_stance']])
dataset_df.head()

,question,answer,answer_stance
171,Unusual choice: Windows XP (x64) vs. Windows S...,Windows Server 2008 have same base as Vista. ...,3
142,Apprenticeship vs. imitation learning - what i...,"In general, yes, they are the same thing, whic...",2
12,Python productivity VS Java Productivity,"One of Python's major advantages is its ""batte...",2
454,To achieve maximum flavour is it better to cut...,Tearing is for artistic effect Some people wi...,2
10,What's the better user experience: Waiting onc...,"Startup as fast as possible, use a thread to d...",3


In [26]:
print(dataset_df.shape)
train_df = dataset_df[:380]
# val_df = dataset_df[335:380]
val_df = dataset_df[380:]
print(train_df.shape)
print(val_df.shape)
# print(test_df.shape)

(445, 3)
(380, 3)
(65, 3)


In [27]:
train_df.head(5)

,question,answer,answer_stance
171,Unusual choice: Windows XP (x64) vs. Windows S...,Windows Server 2008 have same base as Vista. ...,3
142,Apprenticeship vs. imitation learning - what i...,"In general, yes, they are the same thing, whic...",2
12,Python productivity VS Java Productivity,"One of Python's major advantages is its ""batte...",2
454,To achieve maximum flavour is it better to cut...,Tearing is for artistic effect Some people wi...,2
10,What's the better user experience: Waiting onc...,"Startup as fast as possible, use a thread to d...",3


In [28]:
# execute once only, then only it will work correctly.
def replace_labels(val):
    if val == 0 or val == 1:
        return 'NO_OBJECT'
    elif val == 2: 
        return 'FIRST_OBJECT'
    else:
        return 'SECOND_OBJECT'
    

In [29]:
# apply the replace_labels function in all the dataframe values.
train_df['answer_stance'] = train_df['answer_stance'].apply(lambda x : replace_labels(x))
val_df['answer_stance'] = val_df['answer_stance'].apply(lambda x : replace_labels(x))
# test_df['answer_stance'] = test_df['answer_stance'].apply(lambda x : replace_labels(x))
train_df['answer_stance'] = train_df['answer_stance'].astype(str)
val_df['answer_stance'] = val_df['answer_stance'].astype(str)
# test_df['answer_stance'] = test_df['answer_stance'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [32]:
val_df.head()

,question,answer,answer_stance
204,Self-drive vs. guided safari in Southern Africa,I can show you my experience in guided and sel...,FIRST_OBJECT
173,Is it better to use a laptop on battery or on ...,It won't make all that much difference. What w...,NO_OBJECT
154,What is a good ground cover for street frontag...,I found it difficult to mark just one answer a...,FIRST_OBJECT
205,"Is it better to book a car rental in advance, ...",It is definitely better to book a car in advan...,FIRST_OBJECT
546,"SSD raid array, should I pick 0 or 5?",Advantages of RAID 5 in your setup: You can...,NO_OBJECT


In [16]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 6.6 MB 47.7 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
     |████████████████████████████████| 895 kB 76.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [18]:
!pip install jsonlines

In [24]:
import torch
import jsonlines
import random
import os
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_cosine_schedule_with_warmup
from tqdm import tqdm
from typing import List
from sklearn.metrics import f1_score, precision_score, recall_score

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')
batch_size = 1
gpu_batch = 1

Using device "cuda"


In [33]:
class LabelPredictionDataset(Dataset):
    def __init__(self, dataset_df):
        self.samples = []
        label_encodings = {'FIRST_OBJECT': 0, 'NO_OBJECT': 1, 'SECOND_OBJECT': 0}

        for q_, a_, l_ in zip(dataset_df['question'],dataset_df['answer'],dataset_df['answer_stance']):
            self.samples.append({
                'claim': q_,
                'rationale': a_,
                'label': label_encodings[l_]
            })


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

def encode(claim: List[str], rationale: List[str]):
    encoding = tokenizer(claim, rationale, padding=True, truncation=True, max_length=512, return_tensors="pt")
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask

def evaluate(model, dataset):
    model.eval()
    targets = []
    outputs = []
    with torch.no_grad():
        for batch in DataLoader(dataset, batch_size=gpu_batch):
            input_ids, attention_mask = encode(batch['claim'], batch['rationale'])
            logits = model(input_ids.to(device)).logits
            targets.extend(batch['label'].float().tolist())
            outputs.extend(logits.argmax(dim=1).tolist())
    return {
        'macro_f1': f1_score(targets, outputs, zero_division=0, average='macro'),
        'f1': tuple(f1_score(targets, outputs, zero_division=0, average=None)),
        'precision': tuple(precision_score(targets, outputs, zero_division=0, average=None)),
        'recall': tuple(recall_score(targets, outputs, zero_division=0, average=None))
    }

In [34]:
trainset = LabelPredictionDataset(train_df)
devset = LabelPredictionDataset(val_df)

In [35]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli").to(device)

Downloading:   0%|          | 0.00/688 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
optimizer = torch.optim.Adam([
            {'params': model.roberta.parameters(), 'lr': 5e-5},
            {'params': model.classifier.parameters(), 'lr': 1e-4}
        ])
scheduler = get_cosine_schedule_with_warmup(optimizer, 128, 50 * 1)

In [37]:
!mkdir /content/drive/MyDrive/touche-2022-prototyping/saved_models

In [38]:
model_path = '/content/drive/MyDrive/touche-2022-prototyping/saved_models'

In [41]:
for e in range(5): # epoch count has to be changed for ideal performance.
    model.train()
    t = tqdm(DataLoader(trainset, batch_size=1, shuffle=True))
    for i, batch in enumerate(t):
        input_ids, attention_mask = encode(batch['claim'], batch['rationale'])
        outputs = model(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device), labels=batch['label'].long().to(device))
        loss = outputs.loss
        loss.backward()
        if (i + 1) % (1 // 1) == 0:
            optimizer.step()
            optimizer.zero_grad()
            t.set_description(f'Epoch {e}, iter {i}, loss: {round(loss.item(), 4)}')
    scheduler.step()
    checkpoint = {
        'epoch': e + 1,
        'optimizer': optimizer.state_dict()
    }
    # Eval
    train_score = evaluate(model, trainset)
    print(f'Epoch {e} train score:')
    print(train_score)
    dev_score = evaluate(model, devset)
    print(f'Epoch {e} dev score:')
    print(dev_score)

    # Save
    # add model_path for saving the ideal model and save it in separate block execution.
    save_path = os.path.join('./saved_models', f'support_classifier-epoch-{e}-f1-{int(dev_score["macro_f1"] * 1e4)}') 
    os.makedirs(save_path)
    tokenizer.save_pretrained(save_path)
    model.save_pretrained(save_path)

Epoch 0, iter 379, loss: 0.0135: 100%|██████████| 380/380 [01:59<00:00,  3.19it/s]


Epoch 0 train score:
{'macro_f1': 0.9973569813945401, 'f1': (0.9975308641975309, 0.9971830985915493), 'precision': (0.9950738916256158, 1.0), 'recall': (1.0, 0.9943820224719101)}
Epoch 0 dev score:
{'macro_f1': 0.6306818181818182, 'f1': (0.6363636363636365, 0.625), 'precision': (0.7, 0.5714285714285714), 'recall': (0.5833333333333334, 0.6896551724137931)}


Epoch 1, iter 379, loss: 0.0388: 100%|██████████| 380/380 [02:00<00:00,  3.15it/s]


Epoch 1 train score:
{'macro_f1': 1.0, 'f1': (1.0, 1.0), 'precision': (1.0, 1.0), 'recall': (1.0, 1.0)}
Epoch 1 dev score:
{'macro_f1': 0.564176245210728, 'f1': (0.5172413793103449, 0.6111111111111112), 'precision': (0.6818181818181818, 0.5116279069767442), 'recall': (0.4166666666666667, 0.7586206896551724)}


Epoch 2, iter 379, loss: 0.0027: 100%|██████████| 380/380 [02:00<00:00,  3.15it/s]


Epoch 2 train score:
{'macro_f1': 0.9815420058149622, 'f1': (0.982367758186398, 0.9807162534435263), 'precision': (1.0, 0.9621621621621622), 'recall': (0.9653465346534653, 1.0)}
Epoch 2 dev score:
{'macro_f1': 0.5469838981014179, 'f1': (0.49122807017543857, 0.6027397260273972), 'precision': (0.6666666666666666, 0.5), 'recall': (0.3888888888888889, 0.7586206896551724)}


Epoch 3, iter 379, loss: 0.0006: 100%|██████████| 380/380 [02:01<00:00,  3.13it/s]


Epoch 3 train score:
{'macro_f1': 0.9973587449868284, 'f1': (0.9975186104218362, 0.9971988795518207), 'precision': (1.0, 0.994413407821229), 'recall': (0.995049504950495, 1.0)}
Epoch 3 dev score:
{'macro_f1': 0.6150201374081972, 'f1': (0.6031746031746033, 0.6268656716417911), 'precision': (0.7037037037037037, 0.5526315789473685), 'recall': (0.5277777777777778, 0.7241379310344828)}


Epoch 4, iter 379, loss: 0.004: 100%|██████████| 380/380 [02:00<00:00,  3.15it/s]


Epoch 4 train score:
{'macro_f1': 0.9947157637112025, 'f1': (0.995049504950495, 0.9943820224719101), 'precision': (0.995049504950495, 0.9943820224719101), 'recall': (0.995049504950495, 0.9943820224719101)}
Epoch 4 dev score:
{'macro_f1': 0.521263958184842, 'f1': (0.5507246376811594, 0.4918032786885246), 'precision': (0.5757575757575758, 0.46875), 'recall': (0.5277777777777778, 0.5172413793103449)}


In [42]:
# Saved models, in /content/saved_models directory.
!ls saved_models/

support_classifier-epoch-0-f1-4372  support_classifier-epoch-3-f1-4615
support_classifier-epoch-0-f1-6306  support_classifier-epoch-3-f1-6150
support_classifier-epoch-1-f1-4707  support_classifier-epoch-4-f1-5062
support_classifier-epoch-1-f1-5641  support_classifier-epoch-4-f1-5212
support_classifier-epoch-2-f1-4743  support_classifier-epoch-5-f1-4800
support_classifier-epoch-2-f1-5469  support_classifier-epoch-6-f1-5248


In [ ]:
# Stance Prediction Module Design:
# Three stance prediction models will be required.
# 